TensorFlow实现简单的卷积网络

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
import tensorflow as tf
sess=tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
import tensorflow as tf
###定义权重和偏置
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

####卷积层和池化层是要重复使用的
#参数x是输入，w是卷积的参数，[5,5,1,32],前两个数字代表卷积核的尺寸；第三个数字代表有多少个channel,最后一个数字代表卷积核的数量，即提取多少类
#的特征
#strides:代表卷积模板移动的步长
#Padding:代表边界的处理方式，SAME代表让卷积的输入和输出保持同样的尺寸
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

####使用2*2的最大池化，即将2*2的像素将为1*1的像素，最大池化会保留原始像素块中灰度值最高的那一个像素，即保留最显著的特征
def max_pool_2_2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


#定义输入
x=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])
###因为卷积神经网络需要用到空间结构信息，需要将1D的输入向量转为2D的图片结构，即从1*784转为28*28的形式，-1代表样本数量不固定，1代表颜色通道的数量
x_image=tf.reshape(x,[-1,28,28,1])

#定义第一个卷积层
#[5,5,1,32]代表卷积核尺寸大小为5*5,1个颜色通道，32个不同的卷积核，然后使用conv2d函数进行卷积操作，并加上偏置，再使用ReLU激活函数进行非线性处理
#最后使用最大池化函数max_pool_2*2对卷积的输出结果进行池化操作
w_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])
h_conv1=tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
h_pool1=max_pool_2_2(h_conv1)

#定义地二个卷积层（提取64中特征）
w_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])
h_conv2=tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2=max_pool_2_2(h_conv2)

#进行全连接层
#经过两次步长为2*2的最大池化，图片的尺寸从28*28变成了7*7，而第二个卷积层的卷积核数量为64，期输出的tensor尺寸为7*7*64
w_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

#为了减轻过拟合，使用dropout层
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

#最后输出层连接一个softmax层
w_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

#定义损失函数和优化算法
#定义损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv)))

#定义优化算法，选择自适应的Adagrad
train_step=tf.train.AdagradOptimizer(1e-4).minimize(cross_entropy)

#定义准确率
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#初始化全局参数
tf.global_variables_initializer().run()
for i in range(20000):
    batch=mnist.train.next_batch(50)
    if i%100==0:
        train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d, training accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

print("test accuracy %g"% accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))


step 0, training accuracy 0.06
step 100, training accuracy 0.4
step 200, training accuracy 0.58
step 300, training accuracy 0.64
step 400, training accuracy 0.72
step 500, training accuracy 0.74
step 600, training accuracy 0.82
step 700, training accuracy 0.82
step 800, training accuracy 0.84
step 900, training accuracy 0.64
step 1000, training accuracy 0.86
step 1100, training accuracy 0.88
step 1200, training accuracy 0.7
step 1300, training accuracy 0.8
step 1400, training accuracy 0.82
step 1500, training accuracy 0.92
step 1600, training accuracy 0.8
step 1700, training accuracy 0.86
step 1800, training accuracy 0.8
step 1900, training accuracy 0.92
step 2000, training accuracy 0.86
step 2100, training accuracy 0.82
step 2200, training accuracy 0.98
step 2300, training accuracy 0.86
step 2400, training accuracy 0.96
step 2500, training accuracy 0.84
step 2600, training accuracy 0.88
step 2700, training accuracy 0.82
step 2800, training accuracy 0.9
step 2900, training accuracy 0.9

KeyboardInterrupt: 